# Using `astropy.units` and `astropy.constants` to check M31 mass estimates
This notebook walks a check of the erratum for Lehner+ (2015) using the `astropy.units` functionality. 

In [79]:
##Set up matplotlib for display in the browser:
%matplotlib inline
import matplotlib as mpl
mpl.rc("savefig", dpi=150)

In [80]:
##Very most basic/standard imports:
import matplotlib.pyplot as plt
import numpy as np

##Automatic unit tracking
import astropy.units as u
##Costants
import astropy.constants as c

# The problem: the CGM mass in M31
We made an error in Lehner+ (2015) [=LHW15] that required an erratum. The error was in our mass calculation, which went awry. Here we will recalculate the mass in M31's CGM to check our numbers in the erratum.

The total mass of gas within an impact parameter is given:

$M_g = \pi \rho^2 \mu m_H N({\rm Si}) ({\rm Si/H})^{-1}_\odot$.

### Quantities 
#### Column density in M31
We start with the surface density of Si in the inner 50 kpc of M31 from LHW15.

In [81]:
# Column of Si ions (I+II+III+IV)
sigmaSi = 7.4e13*u.cm**-2

#### Impact parameter [kpc] and $\mu$:
We'll define $\rho$ and $\mu$ next:

In [82]:
rho = 50.*u.kpc
mu = 1.3

#### Solar abundance of Si:
Now we're going to need the solar abundance of Si as well. 

ND users could pull this from the `pyND.solarabundance` module. Or from `pyigm`. For now we'll hard-code it at $10^{-4.49}$.

In [83]:
solarSi = 10.**(-4.49)

#### Hydrogen mass...in units of solar masses!
Next we grab the hydrogen nuclear mass using `astropy.constants`. We could do this inline below, but this demonstrates how to make use of the constants functionality explicitly.

We'll print the mass to see what it looks like.

In [84]:
m_H = c.m_p
print(m_H)

  Name   = Proton mass
  Value  = 1.672621777e-27
  Uncertainty  = 7.4e-35
  Unit  = kg
  Reference = CODATA 2010


That's a lot of info! We're also going to be interested in the mass in different units. Here's an example of transforming the units.

In [85]:
print(m_H.to('g'),m_H.to('Msun'))

(<Quantity 1.672621777e-24 g>, <Quantity 8.408937594892163e-58 solMass>)


### Calculating the total gas mass
Last we calculate the total gas mass. Note the use of the `.to('')` functionality to make sure all the distance units are on the same scale and the final result is in solar masses.

In [86]:
Mg = np.pi*rho.to('cm')**2 * mu*m_H.to('Msun')*sigmaSi*(1./solarSi)
print(Mg)

186942124.241 cm2 solMass / cm2


The ${\rm cm}^2$ units don't cancel. They would if we put `sigmaSi` right after the $\rho^2$ term (i.e., put all the ${\rm cm}^2$ together). We can work around this:

In [87]:
#The cm^2 units don't cancel. They would if I'd put sigmaSi right after
#   the rho^2 term. Still, we can work around this:
Mg=Mg.to('Msun')

#### Print the final gas mass:
Now let's make a formatted printing of the result, including our chosen $\rho$.

In [88]:
print('Gas mass within {0:0.0f} of M31: {1:0.2e}'.format(rho,Mg))

Gas mass within 50 kpc of M31: 1.87e+08 solMass



#### Getting the units right the first time:
Above we had an issue that the cm$^2$ terms didn't cancel each other out. If we want that to happen, it appears it helps to put the two quantities with cm$^2$ adjacent to one another:

In [89]:
Mg = np.pi*rho.to('cm')**2 *sigmaSi*mu*m_H.to('Msun')*(1./solarSi)

Mg

<Quantity 186942124.241031 solMass>

In [90]:
print('Gas mass within {0:0.0f} of M31: {1:0.2e}'.format(rho,Mg))

Gas mass within 50 kpc of M31: 1.87e+08 solMass


Remember that this gas mass ultimately scales inversely with metallicity.

```
```
## Mass within $R_{\rm vir}$
The following recalculates the mass assuming a constant surface density of Si (see LHW15 for a more realistic version).

In [91]:
rho = 300.*u.kpc
Mg = np.pi*rho.to('cm')**2 *sigmaSi*mu*m_H.to('Msun')*(1./solarSi)

print('Gas mass within {0:0.0f} of M31: {1:0.2e}'.format(rho,Mg))
print('(Assumes constant surface density.)')

Gas mass within 300 kpc of M31: 6.73e+09 solMass
(Assumes constant surface density.)
